REDUCTION DU NOMBRE DE LIGNES DES DIFFERENTS DATASETS

In [ ]:
import pandas as pd

# Liste des noms de fichiers avec les initiales suivies du chiffre dans leur nom
noms_fichiers = ['boiler_226.csv', 'computer_44.csv', 'dishwasher_53.csv', 'dryer_219.csv', 'fridge_284.csv', 'laptop_289.csv', 'tv_290.csv', 'vacuum_236.csv', 'washing_machine_343.csv']

# Nombre d'observations cibles pour chaque fichier
nombre_observation_cible = 2000

# Boucle sur chaque fichier CSV dans la liste
for nom_fichier in noms_fichiers:
    # Extraire les initiales du fichier de base
    initiales = ''.join([mot[0] for mot in nom_fichier.split('.')[:-1]])

    # Extraire le chiffre du nom de fichier
    chiffre = ''.join(filter(str.isdigit, nom_fichier))

    # Charger le fichier CSV
    df = pd.read_csv(nom_fichier)

    # Convertir la colonne 'timestamp' en type datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Trier le DataFrame par ordre chronologique
    df_sorted = df.sort_values(by='timestamp')

    # Récupérer toutes les valeurs uniques de la colonne 'power'
    valeurs_uniques = df_sorted['power'].unique()

    # Calculer le nombre d'occurrences de chaque valeur
    occurrences_par_valeur = df_sorted['power'].value_counts()

    # Créer une liste pour stocker les lignes sélectionnées
    lignes_selectionnees = []

    # Boucler sur chaque valeur unique et ajouter un nombre approprié de lignes à la liste
    for valeur in valeurs_uniques:
        lignes = df_sorted[df_sorted['power'] == valeur].sample(min(occurrences_par_valeur[valeur], nombre_observation_cible // len(valeurs_uniques)))
        lignes_selectionnees.append(lignes)

    # Concaténer les lignes sélectionnées en un seul DataFrame
    df_echantillon = pd.concat(lignes_selectionnees)

    # Si le nombre d'observations est inférieur à 2000, échantillonner aléatoirement davantage de lignes
    if len(df_echantillon) < nombre_observation_cible:
        df_echantillon = pd.concat([df_echantillon, df_sorted.sample(n=nombre_observation_cible-len(df_echantillon))])

    # Réinitialiser l'index pour avoir une nouvelle indexation à partir de 0
    df_echantillon = df_echantillon.reset_index(drop=True)

    # Spécifier le nom du nouveau fichier CSV
    nom_nouveau_fichier = initiales + '_' + chiffre + '_reduits.csv'

    # Écrire les données échantillonnées dans le nouveau fichier CSV
    df_echantillon.to_csv(nom_nouveau_fichier, index=False)

    print("Les données échantillonnées ont été écrites dans", nom_nouveau_fichier)
